## **1. Install Packages**

In [ ]:
#Need to install if not installed yet
!pip install matplotlib
!pip install scipy
!pip install sklearn
!pip install pandas
!pip install seaborn
!pip install haversine
!pip install -U wxPython 
!pip install google
!pip install google-api-core
!pip install --upgrade google-api-python-client
!pip install google-cloud
!pip install google-cloud-vision
!pip install google.cloud.bigquery
!pip install google.cloud.storage
!pip install google-auth-oauthlib


## **2. Import Modules**

In [3]:
import multiprocessing as mp
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
from scipy.stats import uniform
from scipy.stats import norm
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import plot_confusion_matrix
import csv
import pandas as pd
import time
import os
from datetime import datetime, timedelta
import seaborn as sn
import requests
import json
import haversine as hs
import wx
import webbrowser

from __future__ import print_function
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
import win32api



import DQ2# Own defined


## **3. Setup Parameters**

In [8]:
## Weights
#mu                =0.9
#Accuracy          =0.20
#Precision         =0.07
#Confidence        =0.16
#Completeness      =0.10
#Timeliness        =0.12
#Data_Volume       =0.16
#Data_Redundancy   =0.02
#Concordance       =0.16
#
#Utility           =0.12
#Accessibility     =0.16
#Interpretability  =0.28
#Reputation        =0.12
#Artificiality     =0.20
#Access_Security   =0.12


mu               = 1.0
Waccuracy        = 0.3506311521
Wconfidence      = 0.1880884436
Wconcordance     = 0.1768628272
Wcompleteness    = 0.148093351
Wprecision       = 0.09875987987
Wdata_Redundancy = 0.03756434625

#pcmWeights = [Waccuracy,Wprecision,Wconfidence,Wcompleteness,Wdata_Redundancy,Wconcordance]

#Period
start_time ="2019-12-01 00:00:00"
end_time   ="2019-12-31 23:59:00"

#Period for synthetic dataset : based on real data.
#start_time ="2021-10-05 00:00:00"
#end_time   ="2021-10-07 23:59:00"

#Variable Inicialization
#p=99 # P for the CI used in the confidence calculation IT WAS USED AS DEFAULT FOR ALL TESTS!
#p=95 # P for the CI used in the confidence calculation
p=99.0 # P for the CI used in the confidence calculation 


## **4. Load Data and Clean Datasets**

In [5]:
try:
    
#Read Data from February
    header_CC=["codigoSerial", "fecha", "hora", "fechaHora", "temperatura", "humedad_relativa", "pm1_df", "pm10_df", "pm25_df", "pm1_nova", "pm10_nova", "pm25_nova", "calidad_temperatura", "calidad_humedad_relativa", "calidad_pm1_df", "calidad_pm10_df", "calidad_pm25_df", "calidad_pm1_nova", "calidad_pm10_nova", "calidad_pm25_nova"]
    datatypes_CC={"codigoSerial":np.uint16, "temperatura":np.float16, "humedad_relativa":np.float16, "pm1_df":np.float32, "pm10_df":np.float32, "pm25_df":np.float32, "pm1_nova":np.float32, "pm10_nova":np.float32, "pm25_nova":np.float32}
    path_for_CC_data=DQ2.get_path('*.csv',"Select Citizen Scientist *.csv file")
    #print(path_for_CC_data)
    #df_CC = pd.read_csv(path_for_CC_data, header=None, names=header_CC, usecols=header_CC , dtype=datatypes_CC,parse_dates=["fecha","hora","fechaHora"])
    df_CC = pd.read_csv(path_for_CC_data, header=None, names=header_CC, usecols=header_CC , dtype=datatypes_CC,parse_dates=["fecha","hora","fechaHora"],dayfirst=True)

    #print(df_CC)
    df_CC.sort_values(by=['codigoSerial','fechaHora'],ignore_index=True)
    print("Source path for Citizen Science nodes data: ",path_for_CC_data)
    
    #Data includes January, February and March
    header_SS=["Fecha_Hora","codigoSerial","pm25","calidad_pm25","pm10","calidad_pm10"]
    datatypes_SS={"codigoSerial":np.uint16,"pm25":np.float32,"pm10":np.float32}
    path_for_SS_data=DQ2.get_path('*.csv',"Select SIATA Stations *.csv file")
    #df_SS = pd.read_csv(path_for_SS_data, header=None,names=header_SS, usecols=header_SS , dtype=datatypes_SS,parse_dates=["Fecha_Hora"])
    df_SS = pd.read_csv(path_for_SS_data, header=None,names=header_SS, usecols=header_SS , dtype=datatypes_SS,parse_dates=["Fecha_Hora"],dayfirst=True)
    df_SS.sort_values(by=['codigoSerial','Fecha_Hora'],ignore_index=True)
    print("Source path for Siata Stations data: ",path_for_SS_data)
    
    
    datatypesDistances={"codigoSerial_CC":np.uint16,"codigoSerial_ES":np.uint16,"Distancia_a_ES":np.float16,"codigoSerial_ES2":np.uint16}
    path_for_distance_files=DQ2.get_path('*.csv',"Select the Nodes to Siata Stations distances *.csv file")
    Distances = pd.read_csv(path_for_distance_files, header=0, dtype=datatypesDistances,index_col="codigoSerial_CC")
    print("The distance files was read")
    
        
except:
    print("An exception occurred, it is possible that wrong files were chosen, please run again")



#DATA CLEANING
CC, SS=DQ2.clean_sort_data(df_CC, df_SS)
del df_CC
del df_SS

#CC

Source path for Citizen Science nodes data:  C:\Users\julio\Documents\UDEA\Maestría\DQ in IOT\Datasets\Dataset for testing\Real\CC_PM_December.csv
Source path for Siata Stations data:  C:\Users\julio\Documents\UDEA\Maestría\DQ in IOT\Datasets\Dataset for testing\Real\SS_PM December 2019 till March 2020.csv
The distance files was read
Citizen Scientist:  [1, 2, 3, 4, 5, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 37, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 59, 61, 62, 63, 64, 65, 67, 68, 69, 70, 71, 72, 73, 74, 75, 77, 78, 79, 80, 81, 82, 83, 84, 85, 87, 88, 90, 91, 92, 94, 95, 96, 97, 99, 100, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 124, 125, 126, 127, 128, 129, 130, 131, 133, 134, 136, 137, 138, 139, 140, 141, 142, 143, 145, 146, 149, 150, 151, 152, 153, 154, 156, 157, 158, 160, 161, 162, 165, 166, 167, 168, 169, 170, 172, 173, 174, 175,

## **5. Count the Number of Records**

In [9]:
Records_CC=0
NumberOfNodes=0
for i in CC.keys():
    CCdata=CC[i].loc[(CC[i].fechaHora<=end_time) & (CC[i].fechaHora>=start_time),]
    Records_CC+=len(CCdata)
    if len(CCdata)>0:
        NumberOfNodes+=1
print("Records_CC: ",Records_CC, ", Number of nodes: ",NumberOfNodes)

Records_SS=0
NumberOfStations=0
for i in SS.keys():
    SSdata=SS[i].loc[(SS[i].Fecha_Hora<=end_time) & (SS[i].Fecha_Hora>=start_time),]
    Records_SS+=len(SSdata)
    if len(SSdata)>0:
        NumberOfStations+=1
print("Records_SS: ",Records_SS, ", Number of nodes: ",NumberOfStations)

del Records_SS
del Records_CC
del CCdata
del SSdata
del NumberOfNodes,NumberOfStations


Records_CC:  7524875 , Number of nodes:  219
Records_SS:  14880 , Number of nodes:  20


## **6. DQ Evaluation with Parallelization**

In [10]:
import DQ2# Own defined

#testnumber=1
#cpunumber=6
#print("Test: ",testnumber,". Number of CPUs: ",cpunumber)
t0= time.time()
print("Start time: ", datetime.fromtimestamp(t0))


dim_time = pd.DataFrame(
        columns =["codigoSerial",
                  "fechaHora",
                  "precision_df_time",
                  "precision_nova_time",
                  "uncertainty_time",
                  "accuracy_df_time",
                  "accuracy_nova_time",
                  "completeness_df_time",
                  "completeness_nova_time",
                  "concordance_df_nova_time",
                  "concordance_df_siata_time",
                  "concordance_df_hum_time",
                  "concordance_df_temp_time",
                  "concordance_nova_siata_time",
                  "concordance_nova_hum_time",
                  "concordance_nova_temp_time",
                  "vm_df",
                  "vm_nova",
                  "v",
                  "duplicates_time",
                  
                  "confi_df_time",
                  "confi_nova_time"])

dim_node = pd.DataFrame(
        columns =["codigoSerial",
                  "precision_df_node",
                  "precision_nova_node",
                  "uncertainty_node",
                  "accuracy_df_node",
                  "accuracy_nova_node",
                  "completeness_df_node",
                  "completeness_nova_node",
                  "concordance_df_nova_node",
                  "concordance_df_siata_node",
                  "concordance_df_hum_node",
                  "concordance_df_temp_node",
                  "concordance_nova_siata_node",
                  "concordance_nova_hum_node",
                  "concordance_nova_temp_node",
                  
                  "duplicates_node",
                  
                  "confi_df_node",
                  "confi_nova_node",
                  "DQ_INDEX_NODE"])

dim_DQ = pd.DataFrame(
        columns =["precision_df_total",
                  "precision_nova_total",
                  "uncertainty_total",
                  "accuracy_df_total",
                  "accuracy_nova_total",
                  "completeness_df_total",
                  "completeness_nova_total",
                  "concordance_df_nova_total",
                  "concordance_df_siata_total",#MAYBE NEED TO BE CALCULATED ON A DAILY BASIS
                  "concordance_df_hum_total",
                  "concordance_df_temp_total",
                  "concordance_nova_siata_total",#MAYBE NEED TO BE CALCULATED ON A DAILY BASIS
                  "concordance_nova_hum_total",
                  "concordance_nova_temp_total",
                  
                  "duplicates_total",
                  
                  "confi_df_total",
                  "confi_nova_total",
                  "DQ_INDEX_TOTAL"])

if __name__ ==  '__main__':
    print("Number of avaliable CPUs: ",mp.cpu_count())
    pool=mp.Pool(processes = mp.cpu_count())
    arguments=[]
    #results=pool.map(DQ.eval_dq,[nodes for nodes in CC.keys()])
    results=pool.map(DQ2.eval_dq,([[nodes, CC, SS, Distances, start_time, end_time, p] for nodes in CC.keys()]))
    


    for i in range(0,len(results)):
        dim_time=dim_time.append(results[i][0], ignore_index = True)
        dim_node=dim_node.append(results[i][1], ignore_index = True)

        
    cols =[       "precision_df_node",
                  "precision_nova_node",
                  "uncertainty_node",
                  "accuracy_df_node",
                  "accuracy_nova_node",
                  "completeness_df_node",
                  "completeness_nova_node",
                  "concordance_df_nova_node",
                  "concordance_df_siata_node",
                  "concordance_df_hum_node",
                  "concordance_df_temp_node",
                  "concordance_nova_siata_node",
                  "concordance_nova_hum_node",
                  "concordance_nova_temp_node",
                  
                  "duplicates_node",
                  
                  "confi_df_node",
                  "confi_nova_node"]    
    dim_DQ= dim_node[cols].mean()
    dim_DQ.rename({'precision_df_node':          'precision_df_total', 
                   'precision_nova_node':        'precision_nova_total' , 
                   'uncertainty_node':           'uncertainty_total' , 
                   'accuracy_df_node':           'accuracy_df_total', 
                   'accuracy_nova_node':         'accuracy_nova_total', 
                   'completeness_df_node':       'completeness_df_total', 
                   'completeness_nova_node':     'completeness_nova_total', 
                   'concordance_df_nova_node':   'concordance_df_nova_total', 
                   'concordance_df_siata_node':  'concordance_df_siata_total', 
                   'concordance_df_hum_node':    'concordance_df_hum_total', 
                   'concordance_df_temp_node':   'concordance_df_temp_total', 
                   'concordance_nova_siata_node':'concordance_nova_siata_total', 
                   'concordance_nova_hum_node':  'concordance_nova_hum_total', 
                   'concordance_nova_temp_node': 'concordance_nova_temp_total', 
                   'duplicates_node':            'duplicates_total', 
                   'confi_df_node':              'confi_df_total', 
                   'confi_nova_node':            'confi_nova_total'        }, axis=1, inplace=True)
    
print("End Time: ", datetime.fromtimestamp(time.time()))
t1 = time.time() - t0
print("Elapsed Time: ", t1, " Seconds, or ",t1/60," Minutes")
#print(dim_node)
win32api.Beep(2000, 100)



Start time:  2021-10-22 02:33:37.713836
Number of avaliable CPUs:  6
End Time:  2021-10-22 02:41:18.986695
Elapsed Time:  461.2884838581085  Seconds, or  7.688141397635142  Minutes


## **7. DQ_INDEX by a weighted average function**
The weights come from the Pair-Wise Comparison Matrix

In [11]:
#In case it needs to be calculated by node, not required for our report
#dim_node["DQ_INDEX_NODE"]=  Wprecision*dim_node.loc[:,["precision_df_node","precision_nova_node"]].mean(axis=1)+\
#                            Waccuracy*dim_node.loc[:,["accuracy_df_node","accuracy_nova_node"]].mean(axis=1)+\
#                            Wcompleteness*dim_node.loc[:,["completeness_df_node","completeness_nova_node"]].mean(axis=1)+\
#                            Wconfidence*dim_node.loc[:,["confi_df_node","confi_nova_node"]].mean(axis=1)+\
#                            Wconcordance*dim_node.loc[:,["concordance_df_nova_node","concordance_df_siata_node","concordance_nova_siata_node"]].mean(axis=1)+\
#                            Wdata_Redundancy*dim_node.loc[:,["duplicates_node"]].mean(axis=1)
#
#dim_node["precision_node"]= Wprecision*dim_node.loc[:,["precision_df_node","precision_nova_node"]].mean(axis=1)
#dim_node["accuracy_node"]= Waccuracy*dim_node.loc[:,["accuracy_df_node","accuracy_nova_node"]].mean(axis=1)
#dim_node["completeness_node"]= Wcompleteness*dim_node.loc[:,["completeness_df_node","completeness_nova_node"]].mean(axis=1)
#dim_node["confidence_node"]= Wconfidence*dim_node.loc[:,["confi_df_node","confi_nova_node"]].mean(axis=1)
#dim_node["concordance_node"]= Wconcordance*dim_node.loc[:,["concordance_df_nova_node","concordance_df_siata_node","concordance_nova_siata_node"]].mean(axis=1)
#dim_node["redundancy_node"]= Wdata_Redundancy*dim_node.loc[:,["duplicates_node"]].mean(axis=1)
#dim_node["DQ_INDEX_NODE"]=dim_node[["precision_node","accuracy_node","completeness_node","confidence_node","concordance_node","redundancy_node"]].sum(axis=1)

dim_DQ["DQ_INDEX_TOTAL"]=   Wprecision*dim_DQ[["precision_df_total","precision_nova_total"]].mean()+ \
                            Waccuracy*dim_DQ[["accuracy_df_total","accuracy_nova_total"]].mean()+ \
                            Wcompleteness*dim_DQ[["completeness_df_total","completeness_nova_total"]].mean()+ \
                            Wconfidence*dim_DQ[["confi_df_total","confi_nova_total"]].mean()+ \
                            Wconcordance*dim_DQ[["concordance_df_nova_total","concordance_df_siata_total","concordance_nova_siata_total"]].mean()+ \
                            Wdata_Redundancy*dim_DQ[["duplicates_total"]].mean()
#dim_node.head(6)
#dim_node.loc[dim_node.codigoSerial==49,]

#Need to be removed to avoid the API to crash
dim_time.pop("vm_df")
dim_time.pop("vm_nova")
dim_time.pop("v")
print(dim_DQ)
#dim_time

precision_df_total              0.845001
precision_nova_total            0.864821
uncertainty_total               0.812033
accuracy_df_total               0.561984
accuracy_nova_total             0.558102
completeness_df_total           0.911826
completeness_nova_total         0.898420
concordance_df_nova_total       0.704745
concordance_df_siata_total      0.491380
concordance_df_hum_total        0.411223
concordance_df_temp_total       0.412746
concordance_nova_siata_total    0.484660
concordance_nova_hum_total      0.401279
concordance_nova_temp_total     0.400314
duplicates_total                1.000000
confi_df_total                  0.919721
confi_nova_total                0.933033
DQ_INDEX_TOTAL                  0.725737
dtype: float64


## **8. Export to Google Sheets**

In [12]:
# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
         #"https://www.googleapis.com/auth/spreadsheets"

# The ID and range of a sample spreadsheet.
#SAMPLE_SPREADSHEET_ID = '1gkU9I2EqgJCLm-XemWq_oFenkOcsaHRKwDpO2kK7vd8' #udea account
SPREADSHEET_ID = '1QlPuLYvWaJV6QmOTmkUM3BzuiCvM_8mnuAtvLiEFJaI' #new account
#SAMPLE_RANGE_NAME = 'Class Data!A2:E'

def main():
    """Shows basic usage of the Sheets API.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials2.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    service = build('sheets', 'v4', credentials=creds)

    # Replace undefined data with empty spaces
    
    dim_time.replace(np.nan, '', inplace=True)
    dim_time['fechaHora'] = dim_time['fechaHora'].astype(str)
    dim_time.sort_values(by=['codigoSerial','fechaHora'],ignore_index=True)
    
    dim_node.replace(np.nan, '', inplace=True)
    dim_node.sort_values(by=['codigoSerial'],ignore_index=True)
    
    dim_DQ.replace(np.nan, '', inplace=True)

        
    # Call the Sheets API
    print("Clearing the DQ_TIME sheet in the Google Spreadsheet")
    sheet = service.spreadsheets()
    #request = service.spreadsheets().values().clear(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=range_, body=clear_values_request_body)
    clear_sheet = sheet.values().clear(
        spreadsheetId=SPREADSHEET_ID,
        range='DQ_TIME!A1:Z1000000',
        
    ).execute()
    
    print("Clearing the DQ_NODE sheet in the Google Spreadsheet")
    sheet = service.spreadsheets()
    #request = service.spreadsheets().values().clear(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=range_, body=clear_values_request_body)
    clear_sheet = sheet.values().clear(
        spreadsheetId=SPREADSHEET_ID,
        range='DQ_NODE!A1:Z1000000',
        
    ).execute()
    
    print("Clearing the DQ_TOTAL sheet in the Google Spreadsheet")
    sheet = service.spreadsheets()
    #request = service.spreadsheets().values().clear(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=range_, body=clear_values_request_body)
    clear_sheet = sheet.values().clear(
        spreadsheetId=SPREADSHEET_ID,
        range='DQ_TOTAL!A1:Z1000000',
        
    ).execute()

    print("Exporting Data to Google Sheets")
    sheet = service.spreadsheets()
    write_data = sheet.values().update(
        spreadsheetId=SPREADSHEET_ID,
        valueInputOption='RAW',
        range='DQ_TIME!A1',
        body=dict(
            majorDimension='ROWS',
            values=dim_time.T.reset_index().T.values.tolist())
    ).execute()
    
    sheet = service.spreadsheets()
    write_data = sheet.values().update(
        spreadsheetId=SPREADSHEET_ID,
        valueInputOption='RAW',
        range='DQ_NODE!A1',
        body=dict(
            majorDimension='ROWS',
            values=dim_node.T.reset_index().T.values.tolist())
    ).execute()
    
    sheet = service.spreadsheets()
    write_data = sheet.values().update(
        spreadsheetId=SPREADSHEET_ID,
        valueInputOption='RAW',
        range='DQ_TOTAL!A1',
        body=dict(
            majorDimension='ROWS',
            values=dim_DQ.T.reset_index().T.values.tolist())
    ).execute()
    
    print("Exporting Data Finished")
#
if __name__ == '__main__':
    main()
    win32api.Beep(2000, 100)
    

Clearing the DQ_TIME sheet in the Google Spreadsheet
Clearing the DQ_NODE sheet in the Google Spreadsheet
Clearing the DQ_TOTAL sheet in the Google Spreadsheet
Exporting Data to Google Sheets
Exporting Data Finished


## **9. To Open the Spreadsheet**

In [13]:
#webbrowser.open('https://docs.google.com/spreadsheets/d/1gkU9I2EqgJCLm-XemWq_oFenkOcsaHRKwDpO2kK7vd8/edit?usp=sharing')#udea
webbrowser.open('https://docs.google.com/spreadsheets/d/1QlPuLYvWaJV6QmOTmkUM3BzuiCvM_8mnuAtvLiEFJaI/edit?usp=sharing')#data quality account

True

## **10. To Open the Report**

In [13]:
#webbrowser.open('https://datastudio.google.com/s/ietWLq_iL-8')#"udea account
webbrowser.open('https://datastudio.google.com/s/hy-ZhY6eEfU')#data quality account

True